# LLM Fine-Tuning: Principles and Steps

**Disclaimer:** This reading involves resource-intensive tasks like model training. If you are using limited hardware, some tasks may take longer to complete.

## Introduction

Have you ever wondered how an AI assistant determines the tone of a review or identifies whether feedback is positive or negative? It all starts with preprocessing—the cornerstone of any successful machine learning model.

In machine learning, preprocessing data is an essential step that directly influences your model's performance. Ensuring that your data is clean, structured, and ready for training sets the stage for accurate predictions and better outcomes. Imagine you are preparing a dataset of patient feedback for sentiment analysis in the healthcare industry. These preprocessing steps will ensure your data is clean and ready for fine-tuning a large language model (LLM) to perform sentiment classification tasks.

This reading will take you through the essential steps in preparing datasets for machine learning tasks, focusing on text data for natural language processing (NLP) applications.

By the end of this reading, you will be able to:

- Clean and preprocess text data for machine learning tasks.
- Apply tokenization, text normalization, and missing data handling techniques to ensure your data is ready for model training.
- Organize and split your dataset into appropriate training, validation, and test sets for optimal model performance.
- Fine-tune and evaluate a large language model for specific tasks like sentiment analysis.

## Step-by-step process to fine-tuning

This reading will guide you through the following steps:

- Step 1: Prepare and clean the dataset
- Step 2: Tokenize the data
- Step 3: Fine-tune the model
- Step 4: Evaluate the model

## Step 1: Prepare and clean the dataset

Noisy datasets can degrade model performance. Cleaning ensures consistent input for better predictions. In this step, you'll:

- Remove URLs, hashtags, and special characters.
- Normalize text by converting it to lowercase.

### Code example: Data cleaning and tokenization

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

# Example dataset (replace with your own if you want)
data_dict = {
    "text": [
        " The staff was very kind and attentive to my needs!!! ",
        "The waiting time was too long, and the staff was rude. Visit us at http://hospitalreviews.com",
        "The doctor answered all my questions...but the facility was outdated. ",
        "The nurse was compassionate & made me feel comfortable!! :) ",
        "I had to wait over an hour before being seen. Unacceptable service! #frustrated",
        "The check-in process was smooth, but the doctor seemed rushed. Visit https://feedback.com",
        "Everyone I interacted with was professional and helpful. "
    ],
    "label": ["positive", "negative", "neutral", "positive", "negative", "neutral", "positive"]
}

# Convert to pandas DataFrame
data = pd.DataFrame(data_dict)

# Clean the text
import re

def clean_text(text):
    text = text.lower().strip()  # Convert to lowercase and remove extra spaces
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    return text

# Apply text cleaning
data["cleaned_text"] = data["text"].apply(clean_text)

## Step 2: Tokenize the data

Tokenization converts text into a format models can process. This step uses Hugging Face's tokenizer to transform text into token IDs. Before running this code you may need to install Transformers. Make sure after you do this to refresh your page and then continue to run code.

!pip install transformers datasets scikit-learn torch accelerate

### Code example

In [ ]:
# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Apply tokenization with padding
def tokenize_function(text):
    return tokenizer(text, truncation=True, padding="max_length", max_length=128)

# Apply tokenization
data["tokenized"] = data["cleaned_text"].apply(tokenize_function)

# Extract tokenized features
data["input_ids"] = data["tokenized"].apply(lambda x: x["input_ids"])
data["attention_mask"] = data["tokenized"].apply(lambda x: x["attention_mask"])

# Drop old tokenized column
data = data.drop(columns=["tokenized"])

print(data.head())

## Step 3: Fine-tune the model

Using the tokenized data, you'll fine-tune a pretrained BERT model.

### Code example

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Split into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["text", "cleaned_text"])
test_dataset = test_dataset.remove_columns(["text", "cleaned_text"])

#print(train_dataset)

# Enable dynamic padding for batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    output_dir="./results",
    logging_dir="./logs",
    report_to="none",
    save_strategy="epoch",
    evaluation_strategy="epoch",
)

# Load pre-trained BERT model (3-class classification)
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=3
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,

## Step 4: Evaluate the model

Evaluate the fine-tuned model's accuracy and F1 score on the test set.

### Code example

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Generate predictions
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = test_dataset['label']

# Calculate metrics
accuracy = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='weighted')

print(f"Accuracy: {accuracy}, F1 Score: {f1}")

## Conclusion

Fine-tuning a large language model (LLM) begins with the critical step of preparing your dataset. From cleaning noisy text to tokenizing and splitting your data, each step is vital for ensuring the model's performance is optimized for specific tasks. This reading has provided you with the tools and knowledge to:

- Preprocess text data, ensuring consistency and quality.
- Tokenize and structure your dataset for machine learning models.
- Fine-tune a pretrained model with relevant hyperparameters.
- Evaluate and deploy the model effectively.

With these skills, you are well-equipped to prepare datasets for a variety of natural language processing (NLP) tasks. By following a structured workflow, you can confidently adapt large language models to meet specialized objectives, unlocking the full potential of AI in real-world applications.